In [6]:
#import the dataset from pycaret repository 
from pycaret.datasets import get_data 
import pandas as pd
# boston = get_data('boston') 

df = pd.read_pickle("./full_data_no_missing_combine_selected.pkl")

#import regression module 
from pycaret.regression import * 

#intialize the setup (in Notebook env)
# exp_reg = setup(df, target = 'medv')

#intialize the setup (in Non-Notebook env)
# exp_reg = setup(boston, target = 'medv', html = False)

#intialize the setup (remote runs like Kaggle or GitHub actions)
# exp_reg = setup(boston, target = 'medv', silent = True)

In [9]:
# %tb
train_data = df[:1460]
numeric_features=[

                  'GarageFinish',
                'FullBath',
                'BsmtExposure',
                'ExterQual',
                  '1stFlrSF',
                  'GarageScale',
                  'OpenPorchSF',
                  'GarageArea',
                  'GarageYrBlt',
                  'MasVnrArea',
                  '2ndFlrSF',
                  'BsmtFinSF1',
                  'YearRemodAdd',
                  'TotalFlrSF',
                  'TotalFullBath',
                  'WoodDeckSF',
                  'TotalBsmtSF',
                  'TotRmsAbvGrd',
                  'GrLivArea',
                  'GarageCars',
                  'LotFrontage',
                  'YearBuilt',
                  'TotalSF',

              ]

# ordinal_features_names = ['KitchenQual',
#  'BsmtFinType1',
#  'BsmtQual',
#  'OverallExterScore',
#  'FireplaceQu',
#  'OverallQual',
#  'OverallAbvGrKitchenScore',
#  'OverallHouseScore',
#  'Fireplaces',
#  'GarageFinish',
#  'FullBath',
#  'BsmtExposure',
#  'ExterQual']

# ordinal_features = {f:list(set(train_data[f].values)) for f in ordinal_features_names}

# for k,v in ordinal_features.items():
#     sorted(v)
# print(ordinal_features)

categorical_features=[
                 'Neighborhood_NoRidge',
                 'Foundation_PConc',
                 'GarageType_Detchd',
                 'HeatingQC_Ex',
                 'Neighborhood_NridgHt',
                 'GarageType_Attchd',
                 'Foundation_CBlock',
                 'MasVnrType_None',
                 'SaleType_New',
                 'HeatingQC_TA',
                 'MasVnrType_3',
                 'Exterior1st_VinylSd',
                 'SaleCondition_Partial',
                 'Exterior2nd_VinylSd',
             ]

exp = setup(data = train_data, target = 'SalePrice', #session_id=5541,
              ignore_features=['Id'],
             categorical_features=categorical_features,
#                 ordinal_features=ordinal_features,
              numeric_features=numeric_features,
             )

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:48:13
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
KitchenQual,Categorical
1stFlrSF,Numeric
GarageScale,Numeric
Neighborhood_NoRidge,Categorical
OpenPorchSF,Numeric
Foundation_PConc,Categorical
GarageType_Detchd,Categorical
HeatingQC_Ex,Categorical
GarageArea,Numeric
GarageYrBlt,Numeric


KeyboardInterrupt: Interrupted by user

In [12]:
import numpy as np
train_data = df[:1460]
train_data.SalePrice = np.log(train_data.SalePrice)
numeric_features=[
                'KitchenQual',
                'BsmtFinType1',
                'BsmtQual',
                'OverallExterScore',
                'FireplaceQu',
                'OverallQual',
                'OverallAbvGrKitchenScore',
                'OverallHouseScore',
                'Fireplaces',
                  'GarageFinish',
                'FullBath',
                'BsmtExposure',
                'ExterQual',
                  '1stFlrSF',
                  'GarageScale',
                  'OpenPorchSF',
                  'GarageArea',
                  'GarageYrBlt',
                  'MasVnrArea',
                  '2ndFlrSF',
                  'BsmtFinSF1',
                  'YearRemodAdd',
                  'TotalFlrSF',
                  'TotalFullBath',
                  'WoodDeckSF',
                  'TotalBsmtSF',
                  'TotRmsAbvGrd',
                  'GrLivArea',
                  'GarageCars',
                  'LotFrontage',
                  'YearBuilt',
                  'TotalSF',
                  'OverallBsmtFinishedScore', 
              ]
categorical_features=[
                 'Neighborhood_NoRidge',
                 'Foundation_PConc',
                 'GarageType_Detchd',
                 'HeatingQC_Ex',
                 'Neighborhood_NridgHt',
                 'GarageType_Attchd',
                 'Foundation_CBlock',
                 'MasVnrType_None',
                 'SaleType_New',
                 'HeatingQC_TA',
                 'MasVnrType_3',
                 'Exterior1st_VinylSd',
                 'SaleCondition_Partial',
                 'Exterior2nd_VinylSd',
             ]
exp = setup(data = train_data, target = 'SalePrice', #session_id=5541,
              ignore_features=['Id'],
             categorical_features=categorical_features,
#                 ordinal_features=ordinal_features,
              numeric_features=numeric_features,
            train_size=0.8
             )

 
Setup Succesfully Completed.


,Description,Value
0,session_id,7692
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(1460, 49)"
4,Missing Values,False
5,Numeric Features,34
6,Categorical Features,14
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [13]:
# return best model
# best = compare_models()

# return best model based on MAPE
best = compare_models(sort = 'R2', n_select = 3, fold=2) #default is 'R2'


# compare specific models
# best_specific = compare_models(whitelist = ['dt','rf','xgboost'])

# blacklist certain models
# best_specific = compare_models(blacklist = ['catboost'])

# return top 3 models based on R2
# top3 = compare_models(n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,CatBoost Regressor,0.0079,0.0001,0.0114,0.8802,0.0033,0.0032,1.9954
1,Gradient Boosting Regressor,0.0083,0.0001,0.0119,0.8695,0.0034,0.0034,0.1732
2,Extra Trees Regressor,0.0084,0.0002,0.0123,0.8623,0.0035,0.0034,0.0968
3,Light Gradient Boosting Machine,0.0087,0.0002,0.0124,0.8587,0.0036,0.0035,0.0706
4,Random Forest,0.0085,0.0002,0.0125,0.8562,0.0036,0.0034,0.1282
5,Extreme Gradient Boosting,0.0090,0.0002,0.0130,0.8449,0.0037,0.0036,0.1101
6,Ridge Regression,0.0090,0.0002,0.0135,0.8322,0.0039,0.0036,0.0035
7,Linear Regression,0.0090,0.0002,0.0135,0.8317,0.0039,0.0036,0.0040
8,Bayesian Ridge,0.0089,0.0002,0.0136,0.8300,0.0039,0.0036,0.0068
9,Random Sample Consensus,0.0081,0.0002,0.0139,0.8220,0.0039,0.0033,0.1995


In [17]:
model = create_model('catboost', fold = 2)
tuned_catboost = tune_model(model, optimize = 'RMSLE', fold = 2)
prediction = predict_model(tuned_catboost, data = df[1461:])

In [18]:
prediction.to_csv('submission_v3.csv', index=False)